In [1]:
%pip install gymnasium
%pip install numpy
%pip install pandas
%pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Dict, Box
import numpy as np
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
import random

In [3]:
random.seed(42)
np.random.seed(42)

In [4]:
data = pd.read_csv("Data_tenpercent.csv")
data = data.dropna(subset=["SHOPPER_RATING"])
data["SHOPPER_ID"] = data["SHOPPER_ID"].astype(int)

In [5]:
print(data["DELIVERY_TIME"].min())
print(data["DELIVERY_TIME"].max())

1.0
431.0


In [19]:
grouped = data.groupby("SHOPPER_ID")
worker_ids = list(grouped.groups.keys())
print(grouped.get_group(2388).iloc[0]["DELIVERY_TIME"])

18.0


In [20]:
num_cols_obs = ["DELIVERY_TIME"]

class GigWorkerEnv(gym.Env):
    def __init__(self):
        self.sample_counts = [0 for _ in worker_ids]
        
        self.action_space = Discrete(len(worker_ids))
        self.action_list = [w for w in worker_ids]

        dct = {}
        for col in num_cols_obs:
            dct[col] = Box(low=data[col].min(), high=data[col].max(), shape=())
        
        self.observation_space = Dict(dct)

    def step(self, action): # update the action space
        wid = self.action_list[action]
        sample_count = self.sample_counts[action]
        worker_records = grouped.get_group(wid)
        record = worker_records.iloc[sample_count]
        # record = worker_records.iloc[sample_count % len(worker_records)]
        self.sample_counts[action] += 1

        if self.sample_counts[action] == len(worker_records):
            self.action_space = Discrete(len(self.action_list) - 1)
            del self.action_list[action]
            del self.sample_counts[action]

        observation = {}
        for col in num_cols_obs:
            observation[col] = record[col]
        
        reward = record["SHOPPER_RATING"]
        terminated = len(self.action_list) == 0
        truncated = False
        info = {}

        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.sample_counts = [0 for _ in worker_ids]
        self.action_space = Discrete(len(worker_ids))
        self.action_list = [w for w in worker_ids]
        return {}, {}

In [21]:
env = GigWorkerEnv()
observation, info = env.reset()

episode_over = False
rounds = 0
total_reward = 0
while not episode_over and rounds < 10000:
    rounds = rounds + 1
    action = env.action_space.sample()  # agent policy that uses the observation and info
    # action = 0
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward

    episode_over = terminated or truncated
total_reward /= 10000
print("Total reward:", total_reward)
# always choose 0 => 4.8097
# random choice => 4.662

env.close()

Total reward: 4.66


In [22]:
print(data["SHOPPER_RATING"].mean())

4.816395642971452
